In [1]:
import sqlite3
import pandas as pd

In [2]:
conn = sqlite3.Connection('data.sqlite')
cur = conn.cursor()

GROUP BY and Aggregate Functions
Let's start by looking at some GROUP BY statements to aggregate our data. The GROUP BY clause groups records into summary rows and returns one record for each group. Typically, GROUP BY also involves an aggregate function (COUNT, AVG, etc.). Lastly, GROUP BY can group by one or more columns.

In [3]:
cur.execute(
    """
    SELECT 
    city
    ,COUNT(employeeNumber)
    FROM offices
    JOIN employees
    USING(officeCode)
    GROUP BY city
    ORDER BY count(employeeNumber) DESC
    ;""")

df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,COUNT(employeeNumber)
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Tokyo,2
4,NYC,2


Aliasing
An Alias is a shorthand for a table or column name. Aliases reduce the amount of typing required to enter a query. Generally, complex queries with aliases are easier to read. Aliases are useful with JOIN, GROUP BY, and aggregates (SUM, COUNT, etc.). An Alias only exists for the duration of the query.

You can alias your GROUP BY by specifying the index of our selection order that we want to group by. This is simply written as GROUP BY 1, with the number "1" referring to the first column name that we are selecting.

Additionally, we can also rename our aggregate to a more descriptive name using the AS claus

In [4]:
cur.execute(
    """
    SELECT 
    city
    ,COUNT(employeeNumber) AS numEmployees
    FROM offices
    JOIN employees
    USING(officeCode)
    GROUP BY 1
    ORDER BY numEmployees DESC
    ;""")
df = pd.DataFrame(cur.fetchall())
df.columns = [x[0] for x in cur.description]
df.head()

,city,numEmployees
0,San Francisco,6
1,Paris,5
2,Sydney,4
3,Tokyo,2
4,NYC,2


In [5]:
cur.execute("""SELECT customerName,
               COUNT(customerName) AS number_purchases,
               MIN(amount) AS min_purchase,
               MAX(amount) AS max_purchase,
               AVG(amount) AS avg_purchase,
               SUM(amount) AS total_spent
               FROM customers
               JOIN payments
               USING(customerNumber)
               GROUP BY customerName
               ORDER BY SUM(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

98


,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
0,Euro+ Shopping Channel,13,116208.40,65071.26,55056.844615,715738.98
1,Mini Gifts Distributors Ltd.,9,101244.59,85410.87,64909.804444,584188.24
2,"Australian Collectors, Co.",4,44894.74,82261.22,45146.267500,180585.07
3,Muscle Machine Inc,4,20314.44,58841.35,44478.487500,177913.95
4,"Dragon Souveniers, Ltd.",4,105743.00,44380.15,39062.757500,156251.03


In [6]:
df.tail()

,customerName,number_purchases,min_purchase,max_purchase,avg_purchase,total_spent
93,Royale Belge,4,1128.20,1627.56,7304.295000,29217.18
94,Frau da Collezione,2,17746.26,7612.06,12679.160000,25358.32
95,Atelier graphique,3,14571.44,6066.78,7438.120000,22314.36
96,Auto-Moto Classics Inc.,3,5858.56,9658.74,7184.753333,21554.26
97,Boards & Toys Co.,2,3452.75,4465.85,3959.300000,7918.60


In [7]:
cur.execute("""SELECT city, COUNT(customerNumber) AS number_customers
               FROM customers
               GROUP BY 1
               HAVING COUNT(customerNumber)>=5;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
print(len(df))
df.head()

2


,city,number_customers
0,Madrid,5
1,NYC,5


In [8]:
cur.execute("""SELECT customerName,
               COUNT(amount) AS number_purchases_over_50K
               FROM customers
               JOIN payments
               USING(customerNumber)
               WHERE amount >= 50000
               GROUP BY customerName
               HAVING count(amount) >= 2
               ORDER BY count(amount) DESC;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

94


,customerName,number_purchases_over_50K
0,Euro+ Shopping Channel,13
1,Mini Gifts Distributors Ltd.,9
2,"Tokyo Collectables, Ltd",4
3,Technics Stores Inc.,4
4,Royale Belge,4


In [9]:
conn = sqlite3.Connection('babe_ruth.db')
cur = conn.cursor()

In [12]:
# Return the total number of years that Babe Ruth played professional baseball

cur.execute(
    """
    SELECT 
    COUNT(year) AS number_of_years
    FROM babe_ruth_stats
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

,number_of_years
0,22


In [15]:
# Return the total number of years that Babe Ruth played professional baseball with NY

cur.execute(
    """
    SELECT 
    COUNT(year) AS number_of_years
    ,team
    FROM babe_ruth_stats
    WHERE team = 'NY'
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

,number_of_years,team
0,15,NY


In [33]:
# Select the row with the most HR that Babe Ruth hit in one season

cur.execute(
    """
    SELECT *
    FROM babe_ruth_stats
    ORDER BY HR DESC
    LIMIT 1
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

,id,year,team,league,doubles,triples,hits,HR,games,runs,RBI,at_bats,BB,SB,SO,AVG
0,14,1927,NY,AL,29,8,192,60,151,158,164,540,137,7,89,0.356


In [34]:
cur.execute(
    """
    SELECT
    SUM(HR) AS total_HR_number
    FROM babe_ruth_stats
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df.head()

,total_HR_number
0,714


Five Worst HR Seasons With at Least 100 Games Played
Above you saw that Babe Ruth hit 0 home runs in his first year when he played only five games. To avoid this and other extreme outliers, first filter the data to include only those years in which Ruth played in at least 100 games. Then, select all of the columns for the 5 worst seasons, in terms of the number of home runs, where he played over 100 games.

In [40]:
cur.execute(
    """
    SELECT *
    FROM babe_ruth_stats
    WHERE games >= 100
    ORDER BY HR ASC
    LIMIT 5
    
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df

,id,year,team,league,doubles,triples,hits,HR,games,runs,RBI,at_bats,BB,SB,SO,AVG
0,21,1934,NY,AL,17,4,105,22,125,78,84,365,104,1,63,0.288
1,6,1919,BOS,AL,34,12,139,29,130,103,114,432,101,7,58,0.322
2,20,1933,NY,AL,21,3,138,34,137,97,103,459,114,4,90,0.301
3,9,1922,NY,AL,24,8,128,35,110,94,99,406,84,2,80,0.315
4,10,1923,NY,AL,45,13,205,41,152,151,131,522,170,17,93,0.393


Select the average, AVG, of Ruth's batting averages. The header of the result would be AVG(AVG) which is quite confusing, so provide an alias of career_average.

In [41]:
cur.execute(
    """
    SELECT
    AVG(AVG) AS career_average
    FROM babe_ruth_stats
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df

,career_average
0,0.322864


Select the total number of years played (AS num_years) and total hits (AS total_hits) Babe Ruth had for each team he played for.

In [48]:
cur.execute(
    """
    SELECT
    team
    ,COUNT(year) AS num_years
    ,SUM(hits) AS total_hits
    FROM babe_ruth_stats
    GROUP BY team
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df

,team,num_years,total_hits
0,BOS,7,355
1,NY,15,2518


Number of Years with Over 300 Times On Base
We want to know the years in which Ruth successfully reached base over 300 times. We need to add hits and BB to calculate how many times Ruth reached base. Simply add the two columns together (ie: SELECT [columnName] + [columnName] AS ...) and give this value an alias of on_base. Select the year and on_base for only those years with an on_base over 300.

In [49]:
cur.execute(
    """
    SELECT
    year
    ,hits + BB AS on_base
    FROM babe_ruth_stats
    WHERE on_base > 300
    ;""")
df = pd.DataFrame(cur.fetchall())
df. columns = [i[0] for i in cur.description]
df

,year,on_base
0,1920,322
1,1921,349
2,1923,375
3,1924,342
4,1926,328
5,1927,329
6,1928,310
7,1930,322
8,1931,327
